# Prepare OCIM dataset and visualization

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import hvplot.xarray

### Load OCIM model data

In [14]:
file_path = './data/output_O2_age_new.nc'
ds_all = xr.open_dataset(file_path)
ds_all

<xarray.Dataset> Size: 6MB
Dimensions:        (z: 24, y: 180, x: 91, ylon: 180, x1: 1, ylat: 91, ydepth: 24)
Dimensions without coordinates: z, y, x, ylon, x1, ylat, ydepth
Data variables:
    dO_conc        (z, y, x) float64 3MB ...
    water_age      (z, y, x) float64 3MB ...
    OCIM_grid_lon  float64 8B ...
    OCIM_lon       float64 8B ...
    lon_OCIM       (ylon, x1) float64 1kB ...
    lat_OCIM       (ylat, x1) float64 728B ...
    depth_OCIM     (ydepth, x1) float64 192B ...

#### Clean up data structure

In [3]:
ds_all = ds_all.set_coords(['lat_OCIM', 'lon_OCIM', 'depth_OCIM']) 
ds_all = ds_all.drop_vars({'OCIM_lon','OCIM_grid_lon'}) 
ds_all = ds_all.squeeze('x1', drop=True) 
# Rename dims for consistency 
ds_all = ds_all.rename({'ylon': 'y', 'ylat': 'x', 'ydepth':'z'}) 
# Promote them to be the coordinates 
ds_all = ds_all.assign_coords( y=ds_all['lon_OCIM'], x=ds_all['lat_OCIM'], z=ds_all['depth_OCIM'] ) 
ds_all = ds_all.drop_vars(['lon_OCIM', 'lat_OCIM', 'depth_OCIM']) 
ds_all = ds_all.rename({'x': 'lat', 'y': 'lon', 'z':'depth'})

ds_all

<xarray.Dataset> Size: 6MB
Dimensions:    (depth: 24, lon: 180, lat: 91)
Coordinates:
  * lon        (lon) float64 1kB -179.0 -177.0 -175.0 ... 175.0 177.0 179.0
  * lat        (lat) float64 728B -89.01 -87.03 -85.05 ... 85.05 87.03 89.01
  * depth      (depth) float64 192B 18.07 54.77 93.73 ... 4.825e+03 5.433e+03
Data variables:
    dO_conc    (depth, lon, lat) float64 3MB ...
    water_age  (depth, lon, lat) float64 3MB ...

Change longitude from -180~180 to 0~360

In [4]:
ds_all = ds_all.assign_coords(lon = ((ds_all['lon'] + 180)))

Convert water age unit from (year) to (s) after setting negative values to zero

In [7]:
ds_all['water_age'] = ds_all['water_age'].clip(min=0)

In [8]:
seconds_per_year = 365.25 * 24 * 60 * 60  # days * hours * minutes * seconds
ds_all['water_age_sec'] = ds_all['water_age'] * seconds_per_year

#### save the reconstructed dataset as nc file

In [13]:
ds_all.to_netcdf("./data/ds_OCIM_struct.nc")

### Visualization

#### test at the shallowest layer

In [9]:
ds_slice = ds_all.isel(depth=0)
ds_slice['water_age_sec'].hvplot.image(x='lon', y='lat', cmap='plasma', 
                                 title = f'Water Age (depth={ds_all.depth[0].values:.1f} m)')

:Image   [lon,lat]   (water_age_sec)

#### use hvplot to check layer in different depth

In [10]:
ds_all['dO_conc'].hvplot.image(
    x='lon', y='lat', cmap='viridis', groupby='depth',
    geo=True,
    coastline=True,
    projection=ccrs.PlateCarree(),
    project=True,
    title="Dissolved Oxygen [mmol/(m^3 s)]" 
)

:DynamicMap   [depth]
   :Overlay
      .Image.I     :Image   [lon,lat]   (dO_conc)
      .Coastline.I :Feature   [Longitude,Latitude]

In [11]:
ds_all['water_age_sec'].hvplot.image(
    x='lon', y='lat', cmap='viridis', groupby='depth',
    geo=True,
    coastline=True,
    projection=ccrs.PlateCarree(),
    project=True,
    title="Water Age [s]"
)

:DynamicMap   [depth]
   :Overlay
      .Image.I     :Image   [lon,lat]   (water_age_sec)
      .Coastline.I :Feature   [Longitude,Latitude]